In [5]:
import time
start = time.time()

## Import libraries

In [6]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
# Import pandas as pd
import pandas as pd
import glob
import subprocess

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "C:\Users\M\anaconda3\envs\nfts_nba_extraction_env\python.exe"
  * The NumPy version is: "1.21.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


## Delete old moments data

In [7]:
os.getcwd()

'G:\\My Drive\\Data Science\\Data Science Projects\\NFTs\\GitHub\\nfts_nba_extraction'

In [ ]:
now = time.time()

moments_folder = os.getcwd() + "\\moments_data"

files = [os.path.join(moments_folder, filename) for filename in os.listdir(moments_folder)]
for filename in files:
    if (now - os.stat(filename).st_mtime) > 60:
        try:
            os.remove(filename)
        except OSError:
            pass

## Download new moments' data

In [ ]:
# Download new moments' data
chrome_options = webdriver.ChromeOptions()

#os.getcwd()

prefs = {'download.default_directory' : moments_folder}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)

#driver = webdriver.Chrome(executable_path='chromedriver')
time.sleep(3)

driver.get('https://otmnft.com/moments/')#put here the adress of your page
moments_btn = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/form/div[1]/div/div/div[3]/center/button[2]')
moments_btn.click()
time.sleep(4)

driver.close()

## Read the latest moments data

In [ ]:
# Set the path for the latest moments data
latest_moments_path= moments_folder + '\\moments*.csv'

# Create dict specifying data types for Series and zipcode
data_types = {"Series": "category", "zipcode": str}

date_columns = ['Time Stamp (EST)', 'Date of Moment']

try:
    
    # Read the latest moments file
    list_of_files = glob.glob(latest_moments_path) # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # Read the CSV and assign it to the variable moments
    moments = pd.read_csv(latest_file, dtype= data_types, parse_dates=date_columns)
        
    moments = moments[~moments.Set.str.contains("WNBA|In Her Bag")]

    moments = moments.rename(columns={'Date of Moment': 'date_of_moment', 'Player Name': 'player_name'})

    moments['date_of_moment'] = pd.to_datetime(moments.date_of_moment).dt.tz_localize(None)
    
except pd.io.common.CParserError:
    print("Your data contained rows that could not be parsed.")

In [ ]:
moments['player_name'] = moments['player_name'].str.replace('.', '', regex=True)

In [ ]:
moments['Time Stamp (EST)'] = moments['Time Stamp (EST)'].dt.date

In [ ]:
five_thirty_eight_path = os.getcwd() + "\\five_thirty_eight\\pcapmv.xlsx"

# Read spreadsheet and assign it to survey_responses
five_thirty_eight = pd.read_excel(five_thirty_eight_path, engine='openpyxl')

five_thirty_eight['player'] = five_thirty_eight['player'].str.title()

name_suffixes_replacement = {"Iii": "III", "Ii": "II"}

five_thirty_eight['player'] = five_thirty_eight['player'].replace(name_suffixes_replacement, regex=True)

five_thirty_eight['age'] = five_thirty_eight['age'].str.replace(' years old', '', regex=True)

In [ ]:
market_value_replacement = {"\$": "", "m": ""}

five_thirty_eight['market_value'] = five_thirty_eight['market_value'].replace(market_value_replacement, regex=True)

In [ ]:
import difflib

difflib.get_close_matches

five_thirty_eight.player = five_thirty_eight.player.map(lambda x: difflib.get_close_matches(x, moments.player_name, cutoff=0.8))

five_thirty_eight["player"] = five_thirty_eight["player"].str[0]

five_thirty_eight['player'] = five_thirty_eight['player'].astype(str)

In [ ]:
# Merge five_thirty_eight into moments on the player_name
merged_df = five_thirty_eight.merge(moments, how='right', left_on="player",right_on="player_name")

merged_df = merged_df.drop('player', axis=1)

merged_df['moments_count'] = merged_df.groupby(['player_name'])['category'].transform('count')

merged_df_new= pd.crosstab(merged_df.player_name,merged_df.Series)

merged_df_new = merged_df_new.stack().reset_index().rename(columns={0:'series_count'})

merged_df_new['Series'] = merged_df_new['Series'].map({'1':'CS1', '2':'CS2', '3':'CS3', '4':'CS4'}) 

merged_df_new = merged_df_new.pivot_table(values='series_count', index=['player_name'], columns=['Series'], aggfunc='sum')

merged_df_new.fillna(0, inplace= True)

merged_df_new['player_name'] = merged_df_new.index

merged_df_new.reset_index(drop=True, inplace=True)

In [ ]:
# Merge five_thirty_eight into moments on the player_name
moments_538 = merged_df.merge(merged_df_new,on="player_name")

In [ ]:
moments_538['Low Ask'] = moments_538['Low Ask'].astype(int)

moments_538['cs_per_dollar'] = moments_538['Collector Score']/moments_538['Low Ask']

moments_538['cs_per_dollar'] = moments_538['cs_per_dollar'].round(2)

moments_538['market_cap']= moments_538['Circulation Count']*moments_538['Low Ask']

## Delete old NBA Stats data

In [ ]:
stats_folder = os.getcwd() + "\\stats_data"

files = [os.path.join(stats_folder, filename) for filename in os.listdir(stats_folder)]
for filename in files:
    if (now - os.stat(filename).st_mtime) > 60:
        try:
            os.remove(filename)
        except OSError:
            pass

## Download NBA Stats data

In [ ]:
#os.getcwd()

prefs = {'download.default_directory' : stats_folder}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)

#driver = webdriver.Chrome(executable_path='chromedriver')
time.sleep(3)

driver.get('https://www.nbastuffer.com/2021-2022-nba-player-stats/') #put here the adress of your page
stats_btn = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div/article/div/div/div[3]/div/div[1]/button/span')
stats_btn.click()
time.sleep(4)

driver.close()

## Read the latest NBA Stats data

In [ ]:
# Set the path for the latest stats data
latest_stats_path= stats_folder + '\\NBA*.xlsx'

# Create dict specifying data types for Series and zipcode
#data_types = {"Series": "category", "zipcode": str}

try:
    
    # Read the latest stats file
    list_of_stats_files = glob.glob(latest_stats_path) # * means all if need specific format then *.csv
    latest_stats_file = max(list_of_stats_files, key=os.path.getctime)
    
    cols2skip = [0,2,3,4]  
    cols = [i for i in range(29) if i not in cols2skip]
    
    # Read the CSV and assign it to the variable stats
    stats = pd.read_excel(latest_stats_file, skiprows=[0], usecols=cols, engine='openpyxl')
        
    #stats = stats[~stats.Set.str.contains("WNBA|In Her Bag")]

    stats = stats.rename(columns={'FULL NAME': 'full_name', 'MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor': 'minutes_percentage', 'USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor': 'usage_rate', 'TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions': 'turnover_rate', 'eFG%Effective Shooting PercentageWith eFG%, three-point shots made are worth 50% more than two-point shots made. eFG% Formula=(FGM+ (0.5 x 3PM))/FGA': 'effective_shooting_percentage', 'TS%True Shooting PercentageTrue shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.': 'true_shooting_percentage', 'PPGPointsPoints per game.': 'points_per_game', 'RPGReboundsRebounds per game.': 'rebounds_per_game', 'TRB%Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the player while the player is on the court.': 'total_rebound_percentage', 'APGAssistsAssists per game.': 'assists_per_game', 'AST%Assist PercentageAssist percentage is an estimated percentage of teammate field goals a player assisted while the player is on the court': 'assist_percentage', 'SPGStealsSteals per game.': 'steals_per_game', 'BPGBlocksBlocks per game.': 'blocks_per_game', 'TOPGTurnoversTurnovers per game.': 'turnovers_per_game', 'VIVersatility IndexVersatility index is a metric that measures a player’s ability to produce in points, assists, and rebounds. The average player will score around a five on the index, while top players score above 10': 'versatility_index', 'ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.': 'offensive_rating', 'DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.': 'defensive_rating'})
    
except pd.io.common.CParserError:
    print("Your data contained rows that could not be parsed.")

## Merge the latest NBA Stats data

In [ ]:
stats.full_name = stats.full_name.map(lambda x: difflib.get_close_matches(x, moments_538.player_name, cutoff=0.8))

stats["full_name"] = stats["full_name"].str[0]

stats['full_name'] = stats['full_name'].astype(str)

In [ ]:
# Merge five_thirty_eight into moments on the player_name
moments_538_stats = moments_538.merge(stats, how='right', left_on="player_name",right_on="full_name")

In [ ]:
moments_538_stats = moments_538_stats[moments_538_stats['player_name'].notna()]
moments_538_stats = moments_538_stats.drop('full_name', axis=1)

In [ ]:
accumulated_data_path = os.getcwd() + '\\accumulated_data\\merged_data.csv'

In [ ]:
existing_data = pd.read_csv(accumulated_data_path)

In [ ]:
refreshed_df= pd.concat([existing_data, moments_538_stats], axis=0, ignore_index=False)

In [ ]:
refreshed_df = refreshed_df.drop_duplicates()

In [ ]:
refreshed_df.to_csv(accumulated_data_path, sep=',', index=False)

In [ ]:
end = time.time()
print('\n Success! Your moments data has been updated in', int(end - start), 'seconds.')